In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('wine_quality').getOrCreate()

In [0]:
data_white = spark.read.csv('/FileStore/tables/winequality_white.csv',inferSchema='True',header='true',sep=';')
data_red = spark.read.csv('/FileStore/tables/winequality_red.csv',inferSchema='True',header='true',sep=';')

In [0]:
data_white.head(1)

Out[4]: [Row(fixed acidity=7.0, volatile acidity=0.27, citric acid=0.36, residual sugar=20.7, chlorides=0.045, free sulfur dioxide=45.0, total sulfur dioxide=170.0, density=1.001, pH=3.0, sulphates=0.45, alcohol=8.8, quality=6)]

In [0]:
data_red.head(1)

Out[5]: [Row(fixed acidity=7.4, volatile acidity=0.7, citric acid=0.0, residual sugar=1.9, chlorides=0.076, free sulfur dioxide=11.0, total sulfur dioxide=34.0, density=0.9978, pH=3.51, sulphates=0.56, alcohol=9.4, quality=5)]

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
data_white.columns

Out[7]: ['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [0]:
data_red.columns

Out[8]: ['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [0]:
assembler = VectorAssembler(inputCols=['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol'], outputCol='features')

In [0]:
output_white = assembler.transform(data_white)

In [0]:
output_red = assembler.transform(data_red)

In [0]:
output_white.printSchema()

root
-- fixed acidity: double (nullable = true)
-- volatile acidity: double (nullable = true)
-- citric acid: double (nullable = true)
-- residual sugar: double (nullable = true)
-- chlorides: double (nullable = true)
-- free sulfur dioxide: double (nullable = true)
-- total sulfur dioxide: double (nullable = true)
-- density: double (nullable = true)
-- pH: double (nullable = true)
-- sulphates: double (nullable = true)
-- alcohol: double (nullable = true)
-- quality: integer (nullable = true)
-- features: vector (nullable = true)

In [0]:
from pyspark.ml.classification import RandomForestClassifier

In [0]:
rfc_white = RandomForestClassifier(labelCol='quality',featuresCol='features',numTrees=300)

In [0]:
rfc_red = RandomForestClassifier(labelCol='quality',featuresCol='features',numTrees=300)

In [0]:
final_data_white = output_white.select('features','quality')

In [0]:
final_data_red = output_red.select('features','quality')

In [0]:
final_data_white.printSchema()

root
-- features: vector (nullable = true)
-- quality: integer (nullable = true)

In [0]:
final_data_white.show()

+--------------------+-------+
 features|quality|
+--------------------+-------+
[7.0,0.27,0.36,20...| 6|
[6.3,0.3,0.34,1.6...| 6|
[8.1,0.28,0.4,6.9...| 6|
[7.2,0.23,0.32,8....| 6|
[7.2,0.23,0.32,8....| 6|
[8.1,0.28,0.4,6.9...| 6|
[6.2,0.32,0.16,7....| 6|
[7.0,0.27,0.36,20...| 6|
[6.3,0.3,0.34,1.6...| 6|
[8.1,0.22,0.43,1....| 6|
[8.1,0.27,0.41,1....| 5|
[8.6,0.23,0.4,4.2...| 5|
[7.9,0.18,0.37,1....| 5|
[6.6,0.16,0.4,1.5...| 7|
[8.3,0.42,0.62,19...| 5|
[6.6,0.17,0.38,1....| 7|
[6.3,0.48,0.04,1....| 6|
[6.2,0.66,0.48,1....| 8|
[7.4,0.34,0.42,1....| 6|
[6.5,0.31,0.14,7....| 5|
+--------------------+-------+
only showing top 20 rows

In [0]:
final_data_red.show()

+--------------------+-------+
 features|quality|
+--------------------+-------+
[7.4,0.7,0.0,1.9,...| 5|
[7.8,0.88,0.0,2.6...| 5|
[7.8,0.76,0.04,2....| 5|
[11.2,0.28,0.56,1...| 6|
[7.4,0.7,0.0,1.9,...| 5|
[7.4,0.66,0.0,1.8...| 5|
[7.9,0.6,0.06,1.6...| 5|
[7.3,0.65,0.0,1.2...| 7|
[7.8,0.58,0.02,2....| 7|
[7.5,0.5,0.36,6.1...| 5|
[6.7,0.58,0.08,1....| 5|
[7.5,0.5,0.36,6.1...| 5|
[5.6,0.615,0.0,1....| 5|
[7.8,0.61,0.29,1....| 5|
[8.9,0.62,0.18,3....| 5|
[8.9,0.62,0.19,3....| 5|
[8.5,0.28,0.56,1....| 7|
[8.1,0.56,0.28,1....| 5|
[7.4,0.59,0.08,4....| 4|
[7.9,0.32,0.51,1....| 6|
+--------------------+-------+
only showing top 20 rows

In [0]:
train_data_white,test_data_white = final_data_white.randomSplit([0.7,0.3])
train_data_red,test_data_red = final_data_red.randomSplit([0.7,0.3])

In [0]:
rfc_model_white = rfc_white.fit(train_data_white)

In [0]:
rfc_model_red = rfc_red.fit(final_data_red)

In [0]:
rfc_model_white.featureImportances

Out[24]: SparseVector(11, {0: 0.0264, 1: 0.1674, 2: 0.043, 3: 0.046, 4: 0.0707, 5: 0.0888, 6: 0.0411, 7: 0.1152, 8: 0.0385, 9: 0.0283, 10: 0.3345})

In [0]:
rfc_model_red.featureImportances

Out[25]: SparseVector(11, {0: 0.0414, 1: 0.1174, 2: 0.0402, 3: 0.03, 4: 0.0352, 5: 0.0371, 6: 0.1113, 7: 0.063, 8: 0.0313, 9: 0.1529, 10: 0.34})

In [0]:
rfc_preds_white = rfc_model_white.transform(test_data_white)
rfc_preds_red = rfc_model_red.transform(test_data_red)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
my_binary_eval_white = BinaryClassificationEvaluator(labelCol='quality')
my_binary_eval_red = BinaryClassificationEvaluator(labelCol='quality')

In [0]:
print('Random Forest Clasifier Results - White Wine:')
print(my_binary_eval_white.evaluate(rfc_preds_white))

Random Forest Clasifier Results - White Wine:
1.0

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
my_multiclass_eval_white = MulticlassClassificationEvaluator(labelCol='quality',metricName='accuracy')
my_multiclass_eval_red = MulticlassClassificationEvaluator(labelCol='quality',metricName='accuracy')

In [0]:
rfc_acc_white = my_multiclass_eval_white.evaluate(rfc_preds_white)
rfc_acc_red = my_multiclass_eval_red.evaluate(rfc_preds_red)

In [0]:
rfc_acc_white

Out[33]: 0.5446549391069012

In [0]:
rfc_acc_red

Out[34]: 0.6942148760330579

In [0]:
rfc_preds_white.show()

+--------------------+-------+--------------------+--------------------+----------+
 features|quality| rawPrediction| probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
[3.9,0.225,0.4,4....| 8|[0.0,0.0,0.0,0.81...|[0.0,0.0,0.0,0.00...| 7.0|
[4.2,0.215,0.23,5...| 3|[0.0,0.0,0.0,0.44...|[0.0,0.0,0.0,0.00...| 6.0|
[4.4,0.46,0.1,2.8...| 6|[0.0,0.0,0.0,0.26...|[0.0,0.0,0.0,8.85...| 7.0|
[4.4,0.54,0.09,5....| 7|[0.0,0.0,0.0,0.55...|[0.0,0.0,0.0,0.00...| 6.0|
[4.7,0.145,0.29,1...| 6|[0.0,0.0,0.0,1.22...|[0.0,0.0,0.0,0.00...| 6.0|
[4.8,0.21,0.21,10...| 7|[0.0,0.0,0.0,0.53...|[0.0,0.0,0.0,0.00...| 6.0|
[4.8,0.225,0.38,1...| 6|[0.0,0.0,0.0,0.63...|[0.0,0.0,0.0,0.00...| 6.0|
[4.8,0.29,0.23,1....| 6|[0.0,0.0,0.0,0.73...|[0.0,0.0,0.0,0.00...| 6.0|
[4.9,0.345,0.34,1...| 5|[0.0,0.0,0.0,0.68...|[0.0,0.0,0.0,0.00...| 6.0|
[5.0,0.255,0.22,2...| 6|[0.0,0.0,0.0,0.55...|[0.0,0.0,0.0,0.00...| 6.0|
[5.0,0.27,0.32,4....| 7|[0.0,0.0,0.0,0.49...|[0.0,0.0,0.0,0.00...| 7.0|
[5.0,0.29,0.54,5....| 8|[0.0,0.0,0.0,0.70...|[0.0,0.0,0.0,0.00...| 7.0|
[5.0,0.33,0.16,1....| 6|[0.0,0.0,0.0,0.71...|[0.0,0.0,0.0,0.00...| 6.0|
[5.0,0.33,0.23,11...| 6|[0.0,0.0,0.0,0.54...|[0.0,0.0,0.0,0.00...| 6.0|
[5.1,0.11,0.32,1....| 6|[0.0,0.0,0.0,2.70...|[0.0,0.0,0.0,0.00...| 6.0|
[5.1,0.26,0.33,1....| 7|[0.0,0.0,0.0,0.83...|[0.0,0.0,0.0,0.00...| 6.0|
[5.1,0.29,0.28,8....| 6|[0.0,0.0,0.0,0.50...|[0.0,0.0,0.0,0.00...| 6.0|
[5.1,0.33,0.22,1....| 7|[0.0,0.0,0.0,1.16...|[0.0,0.0,0.0,0.00...| 6.0|
[5.1,0.33,0.22,1....| 7|[0.0,0.0,0.0,1.16...|[0.0,0.0,0.0,0.00...| 6.0|
[5.1,0.35,0.26,6....| 6|[0.0,0.0,0.0,0.51...|[0.0,0.0,0.0,0.00...| 6.0|
+--------------------+-------+--------------------+--------------------+----------+
only showing top 20 rows

In [0]:
rfc_preds_white.filter(rfc_preds_white.quality==8).show()

+--------------------+-------+--------------------+--------------------+----------+
 features|quality| rawPrediction| probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
[3.9,0.225,0.4,4....| 8|[0.0,0.0,0.0,0.81...|[0.0,0.0,0.0,0.00...| 7.0|
[5.0,0.29,0.54,5....| 8|[0.0,0.0,0.0,0.70...|[0.0,0.0,0.0,0.00...| 7.0|
[5.2,0.24,0.45,3....| 8|[0.0,0.0,0.0,1.00...|[0.0,0.0,0.0,0.00...| 6.0|
[5.3,0.3,0.16,4.2...| 8|[0.0,0.0,0.0,0.54...|[0.0,0.0,0.0,0.00...| 6.0|
[5.4,0.45,0.27,6....| 8|[0.0,0.0,0.0,0.51...|[0.0,0.0,0.0,0.00...| 7.0|
[5.6,0.12,0.33,2....| 8|[0.0,0.0,0.0,0.46...|[0.0,0.0,0.0,0.00...| 6.0|
[5.6,0.21,0.4,1.3...| 8|[0.0,0.0,0.0,0.54...|[0.0,0.0,0.0,0.00...| 6.0|
[5.9,0.27,0.29,11...| 8|[0.0,0.0,0.0,0.43...|[0.0,0.0,0.0,0.00...| 6.0|
[6.0,0.26,0.42,5....| 8|[0.0,0.0,0.0,0.72...|[0.0,0.0,0.0,0.00...| 6.0|
[6.1,0.28,0.27,4....| 8|[0.0,0.0,0.0,0.46...|[0.0,0.0,0.0,0.00...| 6.0|
[6.2,0.37,0.24,6....| 8|[0.0,0.0,0.0,0.41...|[0.0,0.0,0.0,0.00...| 7.0|
[6.3,0.29,0.28,4....| 8|[0.0,0.0,0.0,0.38...|[0.0,0.0,0.0,0.00...| 6.0|
[6.5,0.13,0.37,1....| 8|[0.0,0.0,0.0,1.12...|[0.0,0.0,0.0,0.00...| 6.0|
[6.5,0.18,0.34,1....| 8|[0.0,0.0,0.0,0.83...|[0.0,0.0,0.0,0.00...| 6.0|
[6.5,0.3,0.27,4.0...| 8|[0.0,0.0,0.0,0.44...|[0.0,0.0,0.0,0.00...| 7.0|
[6.5,0.35,0.36,0....| 8|[0.0,0.0,0.0,0.80...|[0.0,0.0,0.0,0.00...| 6.0|
[6.5,0.36,0.28,3....| 8|[0.0,0.0,0.0,0.46...|[0.0,0.0,0.0,0.00...| 7.0|
[6.6,0.15,0.32,6....| 8|[0.0,0.0,0.0,0.37...|[0.0,0.0,0.0,0.00...| 6.0|
[6.6,0.15,0.32,6....| 8|[0.0,0.0,0.0,0.37...|[0.0,0.0,0.0,0.00...| 6.0|
[6.6,0.16,0.36,1....| 8|[0.0,0.0,0.0,0.59...|[0.0,0.0,0.0,0.00...| 6.0|
+--------------------+-------+--------------------+--------------------+----------+
only showing top 20 rows

Predictions using Ramdon Forest are quite good! Similar to Linear Regression